In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lecture 3B - Data Integration*

# Table of Contents
* [Lecture 3B - Data Integration*](#Lecture-12---Data-Integration*)
	* &nbsp;
	* [Content](#Content)
	* [Learning Outcomes](#Learning-Outcomes)
	* [Integration of data from multiple sources](#Integration-of-data-from-multiple-sources)
	* [Merging Datasets](#Merging-Datasets)
		* [Database-style DataFrame Merges](#Database-style-DataFrame-Merges)
		* [Merging on Index](#Merging-on-Index)
		* [Concatenating Data Frames](#Concatenating-Data-Frames)
		* [Concatenation on axes](#Concatenation-on-axes)
		* [Updating Columns](#Updating-Columns)
			* [Combining together values within Series or DataFrame columns from different sources](#Combining-together-values-within-Series-or-DataFrame-columns-from-different-sources)


---

### Content

1. dataset merging
2. dataset concatenation
3. dataset value updating


\* Content in this notebook is based on the material in the "Python for Data Analysis" book by Wes McKinney, chapter 7. and material from http://pandas.pydata.org/

### Learning Outcomes

At the end of this lecture, you should be able to:

* describe the inner, outer, left, right join-types for merging dataframes 
* merge different dataframes on indices or common columns
* concatenate dataframes horizontally or vertically
* update values in one dataframe based on values from a similar dataframe 

---

In [ ]:
from IPython.display import HTML, IFrame
IFrame("http://pandas.pydata.org/pandas-docs/dev/merging.html", width=1100, height=500)

## Integration of data from multiple sources

Much of the work in the overall analytics pipeline is spent on data preparation: loading, cleaning, transforming, and rearranging. The total time spent on this task can be up to 90% of the entire analytics project time resources, before any actual useful 'analytics' work is done.

Increasingly datasets from multiple sources must be integrated into a single dataset. This can be a difficult task especially if done manually through Excel-type programs. In many cases it is impossible due to file size, and often undesirable to to the fact that it is difficult to document the process and also impossible to audit and repeat automatically.

Many analytics professionals choose to do ad hoc processing of data from one form to another using a general purpose programming, like Python, Perl, R, or Java, or UNIX text processing tools like sed or awk. 

Fortunately, pandas along with the Python standard library provide you with a high-level, flexible, and high-performance set of core manipulations and algorithms to enable you to integrate and wrangle data into a single source without much trouble.

## Merging Datasets

Data contained in pandas objects can be combined together in a number of built-in ways:

* `pandas.merge` connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.
* `pandas.concat` glues or stacks together objects along an axis (`axis=1` columns, `axis=0` rows).

### Database-style DataFrame Merges

Merge or join operations combine data sets by linking rows using one or more **keys**.
These operations are central to relational databases. 

The `merge` function in pandas is
the main entry point for using these algorithms on your data.


We will begin with simple examples:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from pylab import rcParams

# Set some Pandas options as you like
pd.set_option('max_columns', 30)
pd.set_option('max_rows', 30)

rcParams['figure.figsize'] = 15, 10
rcParams['font.size'] = 20

%matplotlib inline

In [ ]:
df1 = pd.DataFrame(
                    {'name': ['ben', 'ben', 'adam', 'cindy', 'adam', 'adam', 'ben'],
                     'transaction': np.random.randint(1, 50, 7)}
                  )

df2 = pd.DataFrame(
                    {'name': ['adam', 'ben', 'darren'],
                     'age': [33,25,40]}
                )
print(df1)
print('---------------')
print(df2)

Below is an example of a many-to-one merge situation using the `pandas.merge` method; the data in `df1` has multiple rows
labelled 'adam' and 'ben', whereas `df2` has only one row for each value in the key column. Calling
merge with these objects we obtain:

In [ ]:
pd.merge(df1, df2)

Note that we **did not specify** which column to join on. If not specified, merge uses the
**overlapping column names as the keys**. It is however good practice to specify explicitly:

In [ ]:
pd.merge(df1, df2, on='name')

Notice that original indexes cannot be preserved when merging on columns.

If the column names are different in each object, you can specify them separately:

In [ ]:
df3 = pd.DataFrame(
                {'lkey': ['ben', 'ben', 'adam', 'cindy', 'adam', 'adam', 'ben'],
                 'data1': np.random.randint(1, 50, 7)}
                )

df4 = pd.DataFrame(
                {'rkey': ['adam', 'ben', 'darren'],
                 'age': [33,25,40]}
                )

print(df3)
print('---------')
print(df4)

In [ ]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

You probably noticed that the 'cindy' and 'darren' values and associated data are missing from
the result. 

**By default merge does an 'inner' join**; the keys in the result are the **intersection** of the two sets.
Other possible options are 'left', 'right', and 'outer'. 

The outer join takes the
union of the keys, combining the effect of applying both left and right joins:

In [ ]:
print(df1)
print('---------')
print(df2)

In [ ]:
pd.merge(df1, df2, how='outer')

The above merges have been examples of **one-to-many and many-to-one merges**. Sometimes it is necessary to perform **one-to-one merges on indexes**, these we perform on indexes and we will see them later.

Many-to-many merges have well-defined though not necessarily intuitive behaviour.

Here’s an example:

In [ ]:
df1 = pd.DataFrame(
                {'name': ['ben', 'ben', 'adam', 'cindy', 'adam', 'ben'],
                 'transaction_1': range(6)}
                )
df2 = pd.DataFrame(
                {'name': ['adam', 'ben', 'adam', 'ben', 'darren'],
                 'transaction_2': range(5)}
                )
print(df1)
print('---------')
print(df2)

In [ ]:
pd.merge(df1, df2, on='name', how='left')

Many-to-many joins form the **Cartesian product** of the rows. Since there were 3 'ben'
rows in the left DataFrame and 2 in the right one, there are 6 'ben' rows in the result.

The join method only affects the distinct key values appearing in the result. 

By this we mean that if there are unique keys in either left or right hand side, the type of join method will determine if rows with the unique values appear in the final result:

In [ ]:
pd.merge(df1, df2, how='inner')

**Exercises**: For the following exercises, use the 3 data sets below (source http://www.goldpriceoz.com/). The datasets below represent the "Gold Price Annual End of Period" for a selection of currencies.

Create dataframes from the datasets below by highlighting the dataset and right-clicking copy, followed by the execution of the following line: 

df = pd.read_clipboard() 

**Exercise**: Your first task is to merge the Year End Period data with each of the 2 datasets containing the period end price of gold. Call them df_USD and df_AUD.

In [ ]:
#df_y = pd.read_clipboard()
df_y

In [ ]:
#df_USD = pd.read_clipboard()
df_USD

In [ ]:
#df_AUD = pd.read_clipboard()
df_AUD

**Exercise**: Merge df_USD and df_AUD so that only common data to both datasets is preserved in the result.

**Exercise**: Merge df_USD and df_AUD so that all data in df_USD is preserved in the result.

**Exercise**: Merge df_USD and df_AUD so that all data in df_AUD is preserved in the result.

**Exercise**: Merge df_USD and df_AUD so that all data from both datasets is preserved in the result.

**Exercise**: Plot the price of gold for one of the currencies, for each of the years in the dataset using an appropriate figure type.

We can merge with multiple keys. To merge with multiple keys, pass a list of column names:

In [ ]:
left = pd.DataFrame(
                 {'key1': ['foo', 'foo', 'bar'],
                 'key2': ['one', 'two', 'one'],
                 'lval': [1, 2, 3]}
                 )

right = pd.DataFrame(
                  {'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]}
                  )
print(left)
print('---------')
print(right)

In [ ]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

To determine which key combinations will appear in the result depending on the choice
of merge method, **think of the multiple keys as forming an array of tuples to be used
as a single join key**.

When joining columns-on-columns, the **indexes on the passed Data Frame objects are discarded**.

A last issue to consider in merge operations is the treatment of overlapping column
names. While you can address the overlap manually, merge has a suffixes option for specifying strings to append to overlapping
names in the left and right DataFrame objects:

In [ ]:
pd.merge(left, right, on='key1')

Notice the suffixes '_x' and '_y' above which are default. We can explicitly specify them:

In [ ]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

**Exercise**: Given the following:

In [ ]:
df5 = pd.DataFrame(
                 {'key1': ['foo', 'foo', 'bar'],
                 'key2': ['one', 'two', 'one'],
                 'val': [1, 2, 3]}
                 )

df6 = pd.DataFrame(
                  {'key1': ['one', 'one', 'one', 'two'],
                   'key2': ['foo', 'foo', 'bar', 'bar'],
                   'val': [4, 5, 6, 7]}
                  )
print(df5)
print('---------')
print(df6)

Your task is to merge on key1 from df5 and key2 from df6 using a merge type that preserves all unique keys, and renaming overlapping columns with the '_l' and '_r' suffixes.

---

### Merging on Index

In some instances, the merge key or keys in a DataFrame will be found in its index. In this
case, you can pass `left_index=True` or `right_index=True` (or both) to indicate that the
index should be used as the merge key:

In [ ]:
left1 = pd.DataFrame(  {'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                        'value': range(6)})

right1 = pd.DataFrame({'group_val': [3.5, 7]}, 
                        index=['a', 'b'])

print(left1)
print('---------')
print(right1)

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True)

Once again, since the default merge method is to intersect the join keys, you can instead form the
union of them with an outer join:

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

DataFrame has a more **convenient join method for merging by index**. It can also be
used to combine together many DataFrame objects **having the same or similar indexes
but non-overlapping columns**. In this example, by merging on unique indexes, we will be performing **one-to-one merge** operations.


In [ ]:
right2 = pd.DataFrame(
                     { 'group_val' : [10,20]
                      }, index=[1,2]
                     )
print(left1)
print('---------')
print(right2)


In [ ]:
left1.join(right2, how='outer')

## Merge Exercises:

**Exercise**: Read in the child_mortality_rates and adult_mortality_rates datasets and merge them on appropriate variables, using a meaningful merge technique. Perform data cleaning where necessary.

In [ ]:
cm = pd.read_csv('../datasets/child_mortality_rates.csv')


**Exercise**: Generate several plots on the above data. Is there anything interesting?

**Exercise**: Read in the adult_mortality_rate_by_cause dataset and merge it with the above dataset on appropriate variables, using a meaningful merge technique. 

In [ ]:
amc = pd.read_csv('../datasets/adult_mortality_rate_by_cause.csv')


**Exercise**: Finally, read in the total_health_expenditure_by_country_per_year dataset. Attempt to merge it with the above dataset. What are the challenges? How might you work around them? 

In [ ]:
th = pd.read_csv('../datasets/total_health_expenditure_peercent_per_capita_of_gdp_by_country_per_year.csv')


---

### Concatenating Data Frames

Concatenation appends data frames and series objects using the `pandas.concat` method. Data frames can be appended either using the axis=0 option (default) whereby rows are added or using the axis=1, whereby columns are added.

In [ ]:
np.random.randn(3, 4)

In [ ]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

print(df1)
print('----------')
print(df2)

In [ ]:
pd.concat([df1, df2], sort=False)

The concat method appends data frames and is not concerned with creating multiple indexes. If the indexes are relevant to the data frame and it is desirable to have unique indexes, then this can be achieved as follows:

In [ ]:
pd.concat([df1, df2], ignore_index=True, sort=False)

**Exercise**: Create a Dataframe called df5 having 4 random float values, having a column called 'a', so that it can be appended with column 'a' from df1. Write code to concat df5 with df1.

### Concatenation on axes

concat can be used to append on the **column axis**:

In [ ]:
df3 = pd.DataFrame(np.random.randn(2, 3), columns=['e', 'f', 'g'])
df3

In [ ]:
df1

In [ ]:
pd.concat([df1, df3], axis=1)

The `concat` method is as powerful as the merge, having a number of arguments that allow you produce custom made concatenation types. We can specify the join axes which selects the specified rows:

In [ ]:
pd.concat([df1, df3], axis=1 , join_axes=[df1.index[1:3]])

### Updating Columns

#### Combining together values within Series or DataFrame columns from different sources

Another fairly common situation is to have two like-indexed (or similarly indexed) Series or DataFrame objects and needing to “patch” values in one dataframe with values from another dataframe based on matching indices. 

Here is an example:

In [ ]:
df1 = pd.DataFrame([[np.nan, 3., 5.], [-4.6, np.nan, 1], [np.nan, 7., np.nan]])
df2 = pd.DataFrame([[-42.6, np.nan, -8.2], [-5., 1.6, 4]], index=[1, 2])

df1

In [ ]:
df2

Say we wanted to update the values in df1, column 2 with those of df1, column 2.

Our intuition might be to do the following:

In [ ]:
df1[2] = df2[2]
df1

From the result above you will notice that all the values from df2[2] have been copied over to df1[2], and that all the existing values in df1[2] have been overwritten. In cases where the index row in df1[2] was not found in df2[2], the new value was assigned as NaN.

However, this is not what we wanted. We wanted to copy the values from df2[2], but preserve the values in df1[2] that did not exist in df2[2].

Let's try again

In [ ]:
df1 = pd.DataFrame([[np.nan, 3., 5.], [-4.6, np.nan, 1], [np.nan, 7., np.nan]])
df1

The function that we need is called update.

In [ ]:
df1[2].update(df2[2])
df1

Note that update performs its operation inplace.

What if we now only wanted to update NaN values in df1 with the values in df2 and not just perform a blanket update?

This can be achieved using the combine_first method.  

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1[[0,1]].combine_first(df2[[0,1]]) 

Note that this method only takes values from the right DataFrame if they are missing in the left DataFrame.

**Exercise:** Use the datasets below and the command df = pd.read_clipboard() in order to construct dataframes for the exercises below:

**Exercise:** df_USD1 has missing values for the USD and GBP. Populate the missing values with those from the dataframe df_USD2

**Exercise:** df_USD1 has a combination of missing values and erroneous values for the EUR column. Replace all the values in this columns with those that exist in dataframe df_USD2 for this column.